In [2]:
pip install requests beautifulsoup4 python-docx

     -------------------------------------- 62.6/62.6 kB 562.7 kB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 503.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 96.6/96.6 kB 918.3 kB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 925.0 kB/s eta 0:00:00
  Running setup.py install for python-docx: started
  Running setup.py install for python-docx: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: python-docx is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
# Script to Extract Questions and their options

import requests
from bs4 import BeautifulSoup
from docx import Document

# base_url = 'https://www.indiabix.com/aptitude/time-and-work/{}'

base_url = 'https://www.indiabix.com/aptitude/profit-and-loss/{}'
page_numbers = range(1, 30)

document = Document()

for page_number in page_numbers:
    if page_number == 1:
        url = base_url.format('')
    elif page_number < 10:
        url = base_url.format(f'01800{page_number}')
    else:
        url = base_url.format(f'0180{page_number}')
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    questions = soup.find_all('div', class_='bix-div-container')
    for question in questions:
        q_no_elem = question.find('div', class_='bix-td-qno')
        q_text_elem = question.find('div', class_='bix-td-qtxt')
        if q_no_elem and q_text_elem:
            q_no = q_no_elem.text.strip()
            q_text = q_text_elem.text.strip()
            for sup in q_text_elem.find_all('sup'):
                sup_text = sup.text.strip()
                sup.replace_with(f"^{sup_text}")
            p = document.add_paragraph(f"{q_no} {q_text}")
        
        options = question.find_all('div', class_='bix-td-option-val')
        option_letters = ['A', 'B', 'C', 'D', 'E']
        p = document.add_paragraph()
        for i, option in enumerate(options):
            if i < len(option_letters):
                option_no = option_letters[i]
                option_text = option.text.strip()
                for sup in option.find_all('sup'):
                    sup_text = sup.text.strip()
                    sup.replace_with(f"^{sup_text}")
                p.add_run(f"{option_no}) {option_text}\t")
            else:
                p.add_run(f"{option.text.strip()}\t")

document.save('questions.docx')


In [44]:
# Script to extract question number and their correct options

import requests
from bs4 import BeautifulSoup
from docx import Document

base_url = 'https://www.indiabix.com/aptitude/profit-and-loss/{}'
page_numbers = range(1, 30)

document = Document()

for page_number in page_numbers:
    if page_number == 1:
        url = base_url.format('')
    elif page_number < 10:
        url = base_url.format(f'01800{page_number}')
    else:
        url = base_url.format(f'0180{page_number}')
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    questions = soup.find_all('div', class_='bix-div-container')
    for question in questions:
        q_no_elem = question.find('div', class_='bix-td-qno')
        answer_elem = question.find('input', class_='jq-hdnakq')
        if q_no_elem and answer_elem:
            q_no = q_no_elem.text.strip()
            answer = answer_elem['value']
            document.add_paragraph(f"{q_no} {answer}")

document.save('answers.docx')
